this is the prediction price teller for houses in tashkent,


In [53]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.head()

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


In [55]:
df.replace('Договорная',52000,inplace=True)

In [56]:
df['size'] = pd.to_numeric(df['size'], errors='coerce').astype(float)
df['price'] = pd.to_numeric(df['price'], errors='coerce').astype(float)

In [59]:
df=df[df['size']<200]

In [ ]:
df.fillna({'size':np.mean(df['size']), "price":np.mean(df['price'])})

In [ ]:
df.isnull().sum()

In [62]:
from sklearn.model_selection import train_test_split
train_set,test_set=train_test_split(df, test_size=0.2, random_state=42)
X_train=train_set.drop('price',axis=1)
y_train=train_set['price'].copy()
df_test=train_set.drop(['location','district','price'],axis=1)
df_columns=train_set.drop(['location','district','price'],axis=1)
df_cat=train_set['district'].copy()

In [63]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [64]:
def std(dataset):
  Std_scaler=StandardScaler()
  return Std_scaler.fit_transform(dataset)
def encoder(cat_dataset):
  cat_attribs=['district']
  pipeline=ColumnTransformer([
                              ('cat',OrdinalEncoder(),cat_attribs)
  ])
  return pipeline.fit_transform(cat_dataset)
df_num=std(df_test)
district=encoder(X_train)
df_ready=pd.DataFrame(df_num,columns=df_columns.columns, index=X_train.index)
df_labels=pd.DataFrame(district,columns=['district'],index=X_train.index)
df_ready['district']=df_labels['district']

In [66]:
df_ready

,rooms,size,level,max_levels,district
4166,-1.554245,-0.719602,1.477683,1.135906,8.0
6497,1.378534,0.747505,0.135047,-0.781799,11.0
5135,-1.554245,-1.348363,0.135047,-0.398258,6.0
3886,-1.554245,-0.754534,-0.312498,2.286530,6.0
1061,0.400941,0.153676,-0.760044,-0.781799,7.0
...,...,...,...,...,...
5300,0.400941,-0.195635,-1.207589,-0.781799,6.0
5335,0.400941,-0.090842,0.135047,-0.781799,6.0
5504,1.378534,0.747505,-0.312498,-0.398258,2.0
870,-0.576652,-0.824396,0.135047,-0.398258,6.0


In [67]:

from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(df_ready, y_train)

RandomForestRegressor()

In [73]:
prediction=RF_model.predict(df_ready)
prediction

array([37541.63556854, 52749.92      , 24923.57380952, ...,
       59528.57287157, 34324.95666833, 53918.23465423])

In [74]:
pd.DataFrame({'Guesses':prediction, 'Real prices': y_train})

,Guesses,Real prices
4166,37541.635569,36998.0
6497,52749.920000,54000.0
5135,24923.573810,24000.0
3886,33926.566667,33500.0
1061,67578.250000,70000.0
...,...,...
5300,50720.974450,61000.0
5335,41832.031001,35900.0
5504,59528.572872,62000.0
870,34324.956668,37000.0


In [75]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(prediction,y_train)
wrong=np.sqrt(mse)
print(wrong)

389099.0105349144
